In [1]:
import os

import pandas as pd
from pprint import pprint

from pvsamlab.system import System 

# Get a list of all .PAN files (case insensitive) in the specified folder
pan_files_folder = '/Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/modules/ja'

def get_pan_files(folder_path):
    """
    Returns a list of all .PAN and .pan files in the specified folder.

    :param folder_path: Path to the folder to search
    :return: List of .PAN and .pan file paths
    """
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.pan')]

pan_files = get_pan_files(pan_files_folder)


data = []

for pan_file in pan_files:
    for year in range(1998, 2024):
        for modules_per_string in range(27, 33):
            print(f"Running simulation for {os.path.basename(pan_file)} year={year} modules_per_string={modules_per_string}")
            plant = System(met_year=str(year),
                           pan_file=pan_file,
                           modules_per_string=modules_per_string)
            
            voc_max = round(max(plant.model.Outputs.subarray1_voc), 2)
            data.append({"Module":plant.module.model,
                         "Year": year, 
                         "ModulesPerString": modules_per_string, 
                         "VocMax": voc_max})
            print("-" * 50)

# Create a dataframe from the collected data
df = pd.DataFrame(data)
print(df)


Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=27
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=28
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=29
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=30
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=31
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1998 modules_per_string=32
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm).PAN year=1999 modules_per_string=27
--------------------------------------------------
Running simulation for JAM66D45-620LB(3.2+2.0mm)

In [2]:
import os
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed
from pvsamlab.system import System

# Get list of .PAN files
def get_pan_files(folder_path):
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.pan')]

# Function to run a single simulation
def run_simulation(pan_file, year, modules_per_string):
    try:
        plant = System(met_year=str(year),
                       pan_file=pan_file,
                       modules_per_string=modules_per_string)
        voc_max = round(max(plant.model.Outputs.subarray1_voc), 2)
        return {
            "Module": plant.module.model,
            "Year": year,
            "ModulesPerString": modules_per_string,
            "VocMax": voc_max
        }
    except Exception as e:
        return {"Error": str(e), "Module": os.path.basename(pan_file), "Year": year, "ModulesPerString": modules_per_string}

# Entry point
if __name__ == "__main__":
    pan_files_folder = '/Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/modules/ja'
    pan_files = get_pan_files(pan_files_folder)

    # Prepare all jobs
    tasks = [(pan, year, mps)
             for pan in pan_files
             for year in range(1998, 2024)
             for mps in range(27, 33)]

    results = []

    # Parallel execution with up to 8 workers (similar to SAM UI)
    with ProcessPoolExecutor(max_workers=8) as executor:
        future_to_task = {executor.submit(run_simulation, *task): task for task in tasks}
        for future in as_completed(future_to_task):
            result = future.result()
            results.append(result)
            print(f"Finished: {result.get('Module')} Year={result.get('Year')} ModulesPerString={result.get('ModulesPerString')}")

    df = pd.DataFrame(results)
    print(df)


Process SpawnProcess-1:
Process SpawnProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/ihorkoshkin/anaconda3/envs/py313/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Users/ihorkoshkin/anaconda3/envs/py313/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ihorkoshkin/anaconda3/envs/py313/lib/python3.13/concurrent/futures/process.py", line 242, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/ihorkoshkin/anaconda3/envs/py313/lib/python3.13/multiprocessing/queues.py", line 120, in get
    return _ForkingPickler.loads(res)
           ~~~~~~~~~~~~~~~~~~~~~^^^^^
AttributeError: Can't get attribute 'run_simulation' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "/Users/ihorkoshkin/anaconda3/envs/py313/lib/python3.13

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

# ThreadPoolExecutor for parallel in Jupyter

In [3]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from pvsamlab.system import System

# --- SETTINGS ---
pan_files_folder = '/Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/modules/ja'
year_range = range(1998, 2024)
string_sizes = range(27, 33)
num_workers = 8  # Same as SAM UI default

# --- HELPERS ---
def get_pan_files(folder_path):
    """
    Returns a list of all .PAN and .pan files in the specified folder.
    """
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.pan')]

def run_simulation(pan_file, year, modules_per_string):
    """
    Runs a single SAM simulation and extracts VocMax.
    Returns a dictionary with results or error info.
    """
    try:
        plant = System(met_year=str(year),
                       pan_file=pan_file,
                       modules_per_string=modules_per_string)
        voc_max = round(max(plant.model.Outputs.subarray1_voc), 2)
        return {
            "Module": plant.module.model,
            "Year": year,
            "ModulesPerString": modules_per_string,
            "VocMax": voc_max
        }
    except Exception as e:
        return {
            "Error": str(e),
            "Module": os.path.basename(pan_file),
            "Year": year,
            "ModulesPerString": modules_per_string
        }

# --- MAIN EXECUTION ---
pan_files = get_pan_files(pan_files_folder)
tasks = [(pan, year, mps) for pan in pan_files for year in year_range for mps in string_sizes]

results = []

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = {executor.submit(run_simulation, *t): t for t in tasks}
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        print(f"Done: {result.get('Module')} | Year={result.get('Year')} | Strings={result.get('ModulesPerString')}")

# Convert results to DataFrame
df = pd.DataFrame(results)
df.head()


Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=27
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=32
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=28
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=28
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=27
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=31
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=30
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1998 | Strings=29
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=30
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=2000 | Strings=30
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=31
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=29
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=1999 | Strings=32
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=2000 | Strings=28
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=2000 | Strings=29
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=2000 | Strings=27
Done: JAM66D45-620/LB(3.2+2.0mm) | Year=2000 | Strings=31
Done: JAM66D45

,Module,Year,ModulesPerString,VocMax
0,JAM66D45-620/LB(3.2+2.0mm),1998,27,1339.28
1,JAM66D45-620/LB(3.2+2.0mm),1998,32,1587.30
2,JAM66D45-620/LB(3.2+2.0mm),1999,28,1397.90
3,JAM66D45-620/LB(3.2+2.0mm),1998,28,1388.89
4,JAM66D45-620/LB(3.2+2.0mm),1999,27,1347.97


In [4]:
df

,Module,Year,ModulesPerString,VocMax
0,JAM66D45-620/LB(3.2+2.0mm),1998,27,1339.28
1,JAM66D45-620/LB(3.2+2.0mm),1998,32,1587.30
2,JAM66D45-620/LB(3.2+2.0mm),1999,28,1397.90
3,JAM66D45-620/LB(3.2+2.0mm),1998,28,1388.89
4,JAM66D45-620/LB(3.2+2.0mm),1999,27,1347.97
...,...,...,...,...
619,JAM66D45-625/LB(3.2+2.0mm),2023,27,1342.62
620,JAM66D45-625/LB(3.2+2.0mm),2023,29,1442.07
621,JAM66D45-625/LB(3.2+2.0mm),2023,32,1591.25
622,JAM66D45-625/LB(3.2+2.0mm),2023,31,1541.52
